<a href="https://colab.research.google.com/github/ikinikar/IPL_Totals/blob/main/IPLTotals_InPyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [117]:
#Unlike v2 this works in PyTorch and attempts to attain a higher accuracy
from google.colab import drive
drive.mount('/content/gdrive')

!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

!pip install focal_loss_torch

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-2ks77rgp
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-2ks77rgp
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0a71d56e5dce3ff1f0dd2c47c29367629262f527
  Preparing metadata (setup.py) ... done
The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin


In [118]:
#Reading In Data
import numpy as np
import pandas as pd
import torch
torch.cuda.is_available()
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
import seaborn as sns
device = torch.device("cuda")

df = pd.read_csv("/content/gdrive/MyDrive/Data/ipldataset.csv")



In [119]:
#Pre-Processing as per https://www.kaggle.com/code/akashdubey022/data-analysis-and-prediction-of-ipl/notebook with a lot of edits as needed
#Step 1: Removing uneeded columns
df.drop(['full_scorecard', 'man_of_the_match', 'margin', 'place'],axis =1 , inplace = True)
df

,team1,team2,team1_score,team2_score,toss_winner,toss_choice,winner,stadium
0,Chennai Super Kings,Kolkata Knight Riders,131.0,133.0,Kolkata Knight Riders,Field,KKR,Wankhede Stadium
1,Mumbai Indians,Delhi Capitals,177.0,179.0,Delhi Capitals,Field,Capitals,Brabourne Stadium
2,Royal Challengers Bangalore,Punjab Kings,205.0,208.0,Punjab Kings,Field,Punjab,Dr DY Patil Sports Academy
3,Lucknow Super Giants,Gujarat Titans,158.0,161.0,Gujarat Titans,Field,Titans,Wankhede Stadium
4,Rajasthan Royals,Sunrisers Hyderabad,210.0,149.0,Sunrisers Hyderabad,Field,Royals,Maharashtra Cricket Association Stadium
...,...,...,...,...,...,...,...,...
953,Royal Challengers Bangalore,Mumbai Indians,NaN,126.0,Mumbai Indians,Field,Mumbai,M Chinnaswamy Stadium
954,Kings XI Punjab,Rajasthan Royals,221.0,180.0,Rajasthan Royals,Field,Kings,Punjab Cricket Association Stadium
955,Rajasthan Royals,Delhi Daredevils,192.0,87.0,Delhi Daredevils,Field,Royals,Wankhede Stadium
956,Kings XI Punjab,Chennai Super Kings,112.0,116.0,Kings XI Punjab,Bat,Super,Wankhede Stadium


In [120]:
#Removing Null Values
df.iloc[:,:-1]
df.dropna(inplace=True)


In [121]:
#Replacing old team names with the new ones for the sake of consistency
df['team1']=df['team1'].str.replace('Delhi Daredevils','Delhi Capitals')
df['team2']=df['team2'].str.replace('Delhi Daredevils','Delhi Capitals')
df['toss_winner']=df['toss_winner'].str.replace('Delhi Daredevils','Delhi Capitals')

df['team1']=df['team1'].str.replace('Deccan Chargers','Sunrisers Hyderabad')
df['team2']=df['team2'].str.replace('Deccan Chargers','Sunrisers Hyderabad')
df['toss_winner']=df['toss_winner'].str.replace('Deccan Chargers','Sunrisers Hyderabad')

df['team1']=df['team1'].str.replace('Rising Pune Supergiants','Rising Pune Supergiant')
df['team2']=df['team2'].str.replace('Rising Pune Supergiants','Rising Pune Supergiant')
df['toss_winner']=df['toss_winner'].str.replace('Rising Pune Supergiants','Rising Pune Supergiant')

df['team1']=df['team1'].str.replace('Kings XI Punjab','Punjab Kings')
df['team2']=df['team2'].str.replace('Kings XI Punjab','Punjab Kings')
df['toss_winner']=df['toss_winner'].str.replace('Kings XI Punjab','Punjab Kings')

In [122]:
#Replacing Team Names With Quicker Acronyms
df['team1']=df['team1'].str.replace('Delhi Capitals', 'DC')
df['team2']=df['team2'].str.replace('Delhi Capitals', 'DC')
df['winner']=df['winner'].str.replace('Capitals','DC')
df['winner']=df['winner'].str.replace('Daredevils','DC')
df['toss_winner']=df['toss_winner'].str.replace('Delhi Capitals', 'DC')

df['team1']=df['team1'].str.replace('Punjab Kings','PBKS')
df['team2']=df['team2'].str.replace('Punjab Kings','PBKS')
df['winner']=df['winner'].str.replace('Punjab','PBKS')
df['toss_winner']=df['toss_winner'].str.replace('Punjab Kings', 'PBKS')

df['team1']=df['team1'].str.replace('Gujarat Titans','GT')
df['team2']=df['team2'].str.replace('Gujarat Titans','GT')
df['winner']=df['winner'].str.replace('Titans','GT')
df['toss_winner']=df['toss_winner'].str.replace('Gujarat Titans','GT')

df['team1']=df['team1'].str.replace('Rajasthan Royals','RR')
df['team2']=df['team2'].str.replace('Rajasthan Royals','RR')
df['winner']=df['winner'].str.replace('Royals','RR')
df['toss_winner']=df['toss_winner'].str.replace('Rajasthan Royals','RR')

df['team1']=df['team1'].str.replace('Chennai Super Kings','CSK')
df['team2']=df['team2'].str.replace('Chennai Super Kings','CSK')
df['winner']=df['winner'].str.replace('Super','CSK')
df['toss_winner']=df['toss_winner'].str.replace('Chennai Super Kings','CSK')

df['team1']=df['team1'].str.replace('Sunrisers Hyderabad','SRH')
df['team2']=df['team2'].str.replace('Sunrisers Hyderabad','SRH')
df['winner']=df['winner'].str.replace('Sunrisers','SRH')
df['winner']=df['winner'].str.replace('Chargers','SRH')
df['toss_winner']=df['toss_winner'].str.replace('Sunrisers Hyderabad','SRH')

df['team1']=df['team1'].str.replace('Mumbai Indians','MI')
df['team2']=df['team2'].str.replace('Mumbai Indians','MI')
df['winner']=df['winner'].str.replace('Mumbai','MI')
df['toss_winner']=df['toss_winner'].str.replace('Mumbai Indians','MI')

df['team1']=df['team1'].str.replace('Royal Challengers Bangalore','RCB')
df['team2']=df['team2'].str.replace('Royal Challengers Bangalore','RCB')
df['toss_winner']=df['toss_winner'].str.replace('Royal Challengers Bangalore','RCB')

df['team1']=df['team1'].str.replace('Kolkata Knight Riders','KKR')
df['team2']=df['team2'].str.replace('Kolkata Knight Riders','KKR')
df['toss_winner']=df['toss_winner'].str.replace('Kolkata Knight Riders','KKR')

df['team1']=df['team1'].str.replace('Punjab Kings','PBKS')
df['team2']=df['team2'].str.replace('Punjab Kings','PBKS')
df['winner']=df['winner'].str.replace('Kings','PBKS')
df['toss_winner']=df['toss_winner'].str.replace('Punjab Kings','PBKS')

df['team1']=df['team1'].str.replace('Gujarat Lions','GL')
df['team2']=df['team2'].str.replace('Gujarat Lions','GL')
df['winner']=df['winner'].str.replace('Guj','GL')
df['toss_winner']=df['toss_winner'].str.replace('Gujarat Lions','GL')

df['team1']=df['team1'].str.replace('Pune Warriors India','PWI')
df['team2']=df['team2'].str.replace('Pune Warriors India','PWI')
df['winner']=df['winner'].str.replace('Warriors','PWI')
df['toss_winner']=df['toss_winner'].str.replace('Pune Warriors India','PWI')

df['team1']=df['team1'].str.replace('Kochi Tuskers Kerala','KTK')
df['team2']=df['team2'].str.replace('Kochi Tuskers Kerala','KTK')
df['winner']=df['winner'].str.replace('Kochi','KTK')
df['toss_winner']=df['toss_winner'].str.replace('Kochi Tuskers Kerala','KTK')

df['team1']=df['team1'].str.replace('Lucknow Super Giants','LSG')
df['team2']=df['team2'].str.replace('Lucknow Super Giants','LSG')
df['winner']=df['winner'].str.replace('Super','LSG')
df['toss_winner']=df['toss_winner'].str.replace('Lucknow Super Giants','LSG')

df['team1']=df['team1'].str.replace('Rising Pune Supergiant','RPS')
df['team2']=df['team2'].str.replace('Rising Pune Supergiant','RPS')
df['winner']=df['winner'].str.replace('Supergiant','RPS')
df['winner']=df['winner'].str.replace('Supergiants','RPS')
df['winner']=df['winner'].str.replace('CSKgiants','RPS')
df['winner']=df['winner'].str.replace('CSKgiant','RPS')
df['winner']=df['winner'].str.replace('RPSs','RPS')
df['toss_winner']=df['toss_winner'].str.replace('Rising Pune Supergiant','RPS')
df

,team1,team2,team1_score,team2_score,toss_winner,toss_choice,winner,stadium
0,CSK,KKR,131.0,133.0,KKR,Field,KKR,Wankhede Stadium
1,MI,DC,177.0,179.0,DC,Field,DC,Brabourne Stadium
2,RCB,PBKS,205.0,208.0,PBKS,Field,PBKS,Dr DY Patil Sports Academy
3,LSG,GT,158.0,161.0,GT,Field,GT,Wankhede Stadium
4,RR,SRH,210.0,149.0,SRH,Field,RR,Maharashtra Cricket Association Stadium
...,...,...,...,...,...,...,...,...
952,SRH,CSK,147.0,148.0,SRH,Bat,CSK,Rajiv Gandhi International Stadium
954,PBKS,RR,221.0,180.0,RR,Field,PBKS,Punjab Cricket Association Stadium
955,RR,DC,192.0,87.0,DC,Field,RR,Wankhede Stadium
956,PBKS,CSK,112.0,116.0,PBKS,Bat,CSK,Wankhede Stadium


In [123]:
#My own processing steps start here (part 1)
#Adding in a column for the batting first score
bat_first_arr = []
bat_first_team = []
chase_team = []
for index, row in df.iterrows():
  if row['toss_choice'] == 'Field':
    if row['toss_winner'] == row['team1']:
      bat_first_arr.append(row['team2_score'])
      bat_first_team.append(row['team2'])
      chase_team.append(row['team1'])

    if row['toss_winner'] == row['team2']:
      bat_first_arr.append(row['team1_score'])
      bat_first_team.append(row['team1'])
      chase_team.append(row['team2'])

  if row['toss_choice'] == 'Bat':
    if row['toss_winner'] == row['team1']:
      bat_first_arr.append(row['team1_score'])
      bat_first_team.append(row['team1'])
      chase_team.append(row['team2'])

    if row['toss_winner'] == row['team2']:
      bat_first_arr.append(row['team2_score'])
      bat_first_team.append(row['team2'])
      chase_team.append(row['team1'])


df2 = df.assign(batting_first = bat_first_team, chasing = chase_team, bat_first = bat_first_arr)
df2.drop(['team1', 'team2', 'team1_score', 'team2_score', 'stadium'], axis =1 , inplace = True) #Too many matches are played in stadiums with less than 50 IPL matches for stadiums to be a consistent metric

#Removing teams with low match counts
df2 = df2.loc[df2['batting_first'] != 'RPS']
df2 = df2.loc[df2['batting_first'] != 'GL']
df2 = df2.loc[df2['batting_first'] != 'GT']
df2 = df2.loc[df2['batting_first'] != 'PWI']
df2 = df2.loc[df2['batting_first'] != 'KTK']

df2 = df2.loc[df2['chasing'] != 'RPS']
df2 = df2.loc[df2['chasing'] != 'GL']
df2 = df2.loc[df2['chasing'] != 'GT']
df2 = df2.loc[df2['chasing'] != 'PWI']
df2 = df2.loc[df2['chasing'] != 'KTK']

#Removing tied and no results (there are not enough of them to take into account)
df2 = df2.loc[df2['winner'] != 'PWI']
df2 = df2.loc[df2['winner'] != 'Tied']
df2 = df2.loc[df2['winner'] != 'No']

df2

,toss_winner,toss_choice,winner,batting_first,chasing,bat_first
0,KKR,Field,KKR,CSK,KKR,131.0
1,DC,Field,DC,MI,DC,177.0
2,PBKS,Field,PBKS,RCB,PBKS,205.0
4,SRH,Field,RR,RR,SRH,210.0
5,RCB,Field,RCB,KKR,RCB,128.0
...,...,...,...,...,...,...
952,SRH,Bat,CSK,SRH,CSK,147.0
954,RR,Field,PBKS,PBKS,RR,221.0
955,DC,Field,RR,RR,DC,192.0
956,PBKS,Bat,CSK,PBKS,CSK,112.0


In [124]:
#Part 2 of pre-processing
x= df2.drop(['winner'],axis=1)
x = pd.get_dummies(x, ['toss_winner', 'batting_first', 'chasing', 'bat_first'])
print(x)
y = df2['winner']
k = pd.get_dummies(y)
print(k)
real_y = []
for i in y:
  if(i == "CSK"):
    real_y.append(0)
  if(i == "DC"):
    real_y.append(1)
  if(i == "KKR"):
    real_y.append(2)
  if(i == "MI"):
    real_y.append(3)
  if(i == "PBKS"):
    real_y.append(4)
  if(i == "RCB"):
    real_y.append(5)
  if(i == "RR"):
    real_y.append(6)
  if(i == "SRH"):
    real_y.append(7)

print(real_y)

y = torch.tensor(real_y)
print(y)

     bat_first  toss_winner_CSK  toss_winner_DC  toss_winner_KKR  \
0        131.0                0               0                1   
1        177.0                0               1                0   
2        205.0                0               0                0   
4        210.0                0               0                0   
5        128.0                0               0                0   
..         ...              ...             ...              ...   
952      147.0                0               0                0   
954      221.0                0               0                0   
955      192.0                0               1                0   
956      112.0                0               0                0   
957      163.0                0               0                0   

     toss_winner_LSG  toss_winner_MI  toss_winner_PBKS  \
0                  0               0                 0   
1                  0               0                 0   
2        

In [125]:
batch_size = 80
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size = 0.80, shuffle = False)

#Scale the data so that it fits between 0 and 1 --> tends to improve accuracy
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
print(x_train)

x_test = scaler.fit_transform(x_test)
xtest = torch.cuda.FloatTensor(x_test)
ytest = torch.nn.functional.one_hot(y_test, num_classes= 8)

#Getting validation data

xt_r, xv_r, yt_r, yv_r = train_test_split(x_train, y_train, train_size = 0.8, shuffle = False)

xtrain = torch.cuda.FloatTensor(xt_r)
ytrain = torch.nn.functional.one_hot(yt_r, num_classes= 8)

xval = torch.cuda.FloatTensor(xv_r)
yval = torch.nn.functional.one_hot(yv_r, num_classes = 8)

#Putting together the DataLoaders
train_set = torch.utils.data.DataLoader(dataset = list(zip(xtrain,yt_r)) , batch_size = batch_size)
val_set = torch.utils.data.DataLoader(dataset = list(zip(xval, yv_r)), batch_size = int(batch_size/4))
test_set = torch.utils.data.DataLoader(dataset = list(zip(xtest, y_test)) , batch_size = batch_size)


[[0.32653061 0.         0.         ... 0.         0.         0.        ]
 [0.56122449 0.         1.         ... 0.         0.         0.        ]
 [0.70408163 0.         0.         ... 0.         0.         0.        ]
 ...
 [0.47959184 0.         0.         ... 0.         0.         1.        ]
 [0.73979592 0.         0.         ... 0.         1.         0.        ]
 [0.38265306 0.         1.         ... 0.         0.         0.        ]]


In [126]:
#Partially based on https://visualstudiomagazine.com/articles/2020/11/04/pytorch-training.aspx at least at first
class Net(nn.Module):
  def __init__(self, i_size, h1_size, n_classes):
    super(Net, self).__init__()
    self.hidden_one = nn.Linear(i_size, h1_size)
    self.hidden_two = nn.Linear(h1_size, h1_size)
    self.output = nn.Linear(h1_size, n_classes)



  def forward(self, x):
    relu = nn.LeakyReLU()
    p = relu(self.hidden_one(x))
    p = relu(self.hidden_two(p))
    #p = relu(self.hidden_three(p))
    return p

In [127]:
#Create model
net = Net(33, 8, 8)
if torch.cuda.is_available():
  net.cuda()

net = net.to('cuda')



In [128]:
#Hash out loss and optimizer
loss_function = nn.CrossEntropyLoss()
x = 0.01
optimizer = torch.optim.Adam(net.parameters(), lr= x, weight_decay = 0.0001)

In [129]:
#Training The Model **** Inspiration from various articles on https://machinelearningmastery.com/ and https://www.youtube.com/watch?v=kyblxb013Lw
e = 1000
for epoch in range(e):
  for i, (x_batch, y_batch) in enumerate(train_set):
      tl, vl = 0, 0
      x_batch, y_batch = x_batch.cuda(), y_batch.cuda()
      y_pred = net(x_batch)
      optimizer.zero_grad()
      train_loss = loss_function(y_pred, y_batch)
      train_loss.backward()
      optimizer.step()



      tl += train_loss.item()
      for k, (xv_batch, yv_batch) in enumerate(val_set):
        xv_batch, yv_batch = xv_batch.cuda(), yv_batch.cuda()
        yv_pred = net(xv_batch)

        val_loss = loss_function(yv_pred, yv_batch)
        vl += val_loss.item()

      if(i % 6 == 0 and i != 0):
        # Did not include the step because I am only printing every final/6th step
        print('Epoch [%d/%d], Loss: %.4f, Val_loss: %.4f' %(epoch + 1, e, tl, vl))


Epoch [1/1000], Loss: 2.0724, Val_loss: 14.5231
Epoch [2/1000], Loss: 2.0508, Val_loss: 14.4442
Epoch [3/1000], Loss: 2.0253, Val_loss: 14.3581
Epoch [4/1000], Loss: 1.9949, Val_loss: 14.2400
Epoch [5/1000], Loss: 1.9503, Val_loss: 14.0210
Epoch [6/1000], Loss: 1.8840, Val_loss: 13.6451
Epoch [7/1000], Loss: 1.7988, Val_loss: 13.2315
Epoch [8/1000], Loss: 1.7026, Val_loss: 12.7251
Epoch [9/1000], Loss: 1.5911, Val_loss: 12.0476
Epoch [10/1000], Loss: 1.4735, Val_loss: 11.3557
Epoch [11/1000], Loss: 1.3564, Val_loss: 10.6929
Epoch [12/1000], Loss: 1.2615, Val_loss: 10.1337
Epoch [13/1000], Loss: 1.1696, Val_loss: 9.5355
Epoch [14/1000], Loss: 1.0577, Val_loss: 8.8681
Epoch [15/1000], Loss: 0.9553, Val_loss: 8.3866
Epoch [16/1000], Loss: 0.8638, Val_loss: 7.9598
Epoch [17/1000], Loss: 0.7866, Val_loss: 7.4442
Epoch [18/1000], Loss: 0.7141, Val_loss: 6.9731
Epoch [19/1000], Loss: 0.6521, Val_loss: 6.6415
Epoch [20/1000], Loss: 0.6033, Val_loss: 6.3133
Epoch [21/1000], Loss: 0.5690, Val_lo

In [130]:
#Evaluating Accuracy of Model
correct = 0
num = 0
net.eval()
with torch.no_grad():
  for a, (xt_batch, yt_batch) in enumerate(test_set):
      xt_batch = xt_batch.cuda()
      yt_batch = yt_batch.cuda()
      yt_pred = net(xt_batch)

      _, predicted = torch.max(yt_pred.data, 1)
      correct += (predicted == yt_batch).sum().item()
      num += (yt_batch.size(0))

print("Model Accuracy: %.3f%%" %(100*correct/num))

Model Accuracy: 61.212%
